In [1]:
import numpy as np
# import folium
# from folium.plugins import MarkerCluster
import pandas as pd
import geopandas as gpd
from fuzzywuzzy import process
from itables import show
import gzip

In [2]:
# load CANESCA data
def read_csv():
    can = pd.read_csv('data/canesca.csv')
    return can

# load country boundaries
def read_gpd():
    can = gpd.read_file('data/geodf.geojson')
    return can

# load state boundaries
adm1_10 = gpd.read_file('data/adm1_10_states.shp')


can = gpd.read_file('data/canesca.csv')
can['Country'].unique()

gdf = read_gpd()
gdf = gdf[['NAME','geometry']]


In [3]:
can.drop(columns=['geometry']).to_csv('data/canesca.csv', index=False)

KeyError: "['geometry'] not found in axis"

## Make Canesca into geojson

In [4]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point


df = pd.read_csv('data/canesca.csv')

In [5]:
def get_points(row):
    print(row['OBJECTID'], row['Latitude'], row['Longitude'])
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        lat = row['Latitude']
        lon = row['Longitude']
        prim = row['PRIMARY_ORGANISATION']
        try:
            point = Point(lon, lat)
            return point
        except:
            print(f"Error with",row['OBJECTID'])
            print(f"Latitude: {lat}")
            print(f"Longitude: {lon}")

df['geometry'] = df.apply(get_points, axis=1)

1 -26.48611191 31.42031691
2 -26.50004655 31.36592668
3 -26.33214699 31.14240787
4 -1.284949908 36.8197854
5 0.512993217 35.28069153
6 -0.521875582 37.4533067
7 -0.447824958 39.6527331
8 -1.856431558 36.77184701
9 -0.085863775 34.77351151
10 -0.669891107 34.7712069
11 -0.088699722 34.77173755
12 -0.03495777 34.75533002
13 -0.110852202 34.75956821
14 -2.27633888 40.9072484
15 -1.526387179 37.26437604
16 -4.047911671 39.6740242
17 -3.21405838 38.5308784
18 -4.069815358 39.67051438
19 -4.064006612 39.68017947
20 -0.717938464 37.1609692
21 -1.264697997 36.81748845
22 -1.309915861 36.8046362
23 -1.25522639 36.83228246
24 -1.300022571 36.80527471
25 -1.30035709 36.80720443
26 -1.176632435 36.91551879
27 -1.287976446 36.79313877
28 -1.272893462 36.89890473
29 -1.306218105 36.83437647
30 -1.307831377 36.80350979
31 -1.290259087 36.81462538
32 -1.262281547 36.81235465
33 -1.259745061 36.84685497
34 -1.286439345 36.79418882
35 0.055146167 36.3631039
36 -1.279548664 36.84556309
37 -1.260257197 36

In [6]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [7]:
gdf.to_file('static/geojson/canesca.geojson', driver='GeoJSON')

/opt/homebrew/Caskroom/miniforge/base/envs/nofrills/lib/python3.12/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


# Class based workflow

In [8]:
import geopandas as gpd
import pandas as pd

class DataMerger:
    def __init__(self, data_110, data_50, data_path, iso_col=None, country_col=None, data_col=None):
        self.NE_110 = gpd.read_file(data_110)
        self.NE_50 = gpd.read_file(data_50)
        self.iso_col = iso_col
        self.country_col = country_col
        self.data_col = data_col
        self.merge_col = self.iso_col if self.iso_col else self.country_col
        self.NEmerge_col = 'ADM0_A3' if self.iso_col else 'NAME_LONG'        
        self.data = self.load_data(data_path)
        self.missing_data = None
        print('NB Make sure zero values represent actual data, not missing data. They will be represented as zero on the map.')

    def load_data(self, data_path):
        data = pd.read_csv(data_path)

        ## replace empty strings and null values with NaN, drop rows with missing data from the input
        data.replace('', np.nan, inplace=True)
        
        if self.data_col and self.data_col in data.columns:
            data = data[data[self.data_col].notna()]
        return data

    def merge_data(self):
        """
        Merge specialist data with 110m data as a base, then attempt to refine with 50m data.
        Additionally, report any specialist data that fails to match with either dataset.
        """
        if len(self.data[self.data[self.merge_col].isna()]) > 0:
              self.missing_data = True
              raise ValueError(f"Missing values in the column {self.merge_col} of the input data. Proceeding means this data will be lost. Add the missing values and try again.")

        # Outer join to include all geometries from 110m and all specialist data
        merged_110 = self.NE_110.merge(self.data, left_on=self.NEmerge_col, right_on=self.merge_col, indicator='merge110', how='outer')
        
        # Get all specialist data matched to the more detailed 50m dataset
        matched_50 = self.NE_50.merge(self.data, how='left', left_on=self.NEmerge_col, right_on=self.merge_col, indicator='merge50')
        
        # Merge the unmatched 110m data with the matched 50m data
        missing_from_110 = merged_110[merged_110['merge110'] == 'right_only']
        missing110_present50 = missing_from_110.merge(matched_50, left_on=self.merge_col, right_on=self.merge_col, how='inner', indicator='final_merge')
        
        # Remove _x columns, these are the 110m columns without data
        missing110_present50 = missing110_present50.drop([col + '_x' for col in matched_50.columns if col + '_x' in missing110_present50.columns], axis=1)
        missing110_present50.columns = [col.replace('_y', '') for col in missing110_present50.columns]
        
        # Concatenate the adjusted 50m data with the 110m data that successfully matched
        merged_110_onlypresent = merged_110[merged_110['merge110'] != 'right_only']
        merged = pd.concat([merged_110_onlypresent, missing110_present50], ignore_index=True)

        # Identify and report countries from the specialist data that did not match any geographic data
        unmatched_specialist_data = self.data.merge(merged[self.merge_col], on=self.merge_col, how='left', indicator=True)
        unmatched_specialist_data = unmatched_specialist_data[unmatched_specialist_data['_merge'] == 'left_only']

        # Assuming that NE_110 and NE_50 have been concatenated and cleaned into a dataframe called geo_data for simplicity
        geo_names = pd.concat([self.NE_110, self.NE_50])['NAME_LONG'].unique()

        # Add a column for fuzzy matching results
        unmatched_specialist_data['Fuzzy_Match'] = unmatched_specialist_data[self.country_col].apply(
            lambda x: process.extractOne(x, geo_names, score_cutoff=80)
        )

        # Clean up the Fuzzy_Match column to show only the matched name, not the tuple with the score
        unmatched_specialist_data['NE_Fuzzy_Match'] = unmatched_specialist_data['Fuzzy_Match'].apply(
            lambda x: x[0] if x is not None else None
        )

        # add the iso code for the matched name
        unmatched_specialist_data['NE_ISO'] = unmatched_specialist_data['NE_Fuzzy_Match'].map(
            self.NE_110.set_index('NAME_LONG')['ADM0_A3'].to_dict()
        )

        #sort unmatched data columns
        unmatched_specialist_data = unmatched_specialist_data[[col for col in [self.iso_col,self.country_col,] if col] + ['NE_Fuzzy_Match','NE_ISO']]

        return merged, unmatched_specialist_data
    
    def replace_country_names(self, replace_dict):
        """
        Replace country names in the specialist data based on a dictionary of replacements.

        Parameters:
        replace_dict (dict): Dictionary mapping original names to new names.
        """
        self.data[self.merge_col] = self.data[self.merge_col].replace(replace_dict)

data_110 = 'data/countries_small/ne_110m_admin_0_countries.shp'
data_50 = 'data/countries_medium/ne_50m_admin_0_countries.shp'

In [9]:
def save_geojson_versions(df, filename):
    # Define the base directory and filename
    base_dir = 'static/geojson/'
    
    # Uncompressed GeoJSON
    uncompressed_path = f'{base_dir}{filename}.geojson'
    df.to_file(uncompressed_path, driver='GeoJSON')
    print(f"Uncompressed GeoJSON saved to {uncompressed_path}")
    
    # Compressed GeoJSON
    compressed_path = f'{base_dir}{filename}.geojson.gz'
    # Convert GeoDataFrame to JSON string
    json_str = df.to_json()
    # Use gzip to compress and write the JSON string to a file
    with gzip.open(compressed_path, 'wt', encoding='utf-8') as gz_file:
        gz_file.write(json_str)
    print(f"Compressed GeoJSON saved to {compressed_path}")

# WFNS (class based)

# 

In [10]:

merger = DataMerger(data_110, data_50, 'data/wfns.csv', iso_col='country_code', country_col='country')


replacedict = {
    'SSD':'SDS',
    'XKX':'KOS',
    'PSE':'PSX'    
}
merger.replace_country_names(replacedict)
merge,unmatched = merger.merge_data()
unmatched

NB Make sure zero values represent actual data, not missing data. They will be represented as zero on the map.


,country_code,country,NE_Fuzzy_Match,NE_ISO
20,CHI,Channel Islands,Falkland Islands / Malvinas,FLK
21,GIB,Gibraltar,None,NaN


In [11]:

merge[merge['NAME_LONG']=='Faeroe Islands']

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,Region,type,income_fy23,income_broad_fy23,workforce_2022,population_2021,density_2022,merge110,merge50,final_merge
189,Admin-0 country,3.0,6.0,Denmark,DN1,1.0,2.0,Dependency,1,Faroe Islands,...,Europe,territory (Denmark),H,HIC,0.0,49053.0,0.0,right_only,both,both


In [12]:
merge[['NAME_LONG','geometry','workforce_2022','population_2021','density_2022']].pipe(save_geojson_versions, 'wfns')

Uncompressed GeoJSON saved to static/geojson/wfns.geojson
Compressed GeoJSON saved to static/geojson/wfns.geojson.gz


# OHNS (class)

In [13]:
ohnsmerger = DataMerger(data_110, data_50, 'data/ohns.csv', country_col='Country Name', data_col='Oral, Head, & Neck Surgeons')

replacedict = {
    'Eswatini':'Kingdom of eSwatini',
        'South Korea':'Republic of Korea',
        'Gambia':'The Gambia',
                 'Cabo Verde':'Republic of Cabo Verde',
                    'Congo DRC':'Democratic Republic of the Congo',
                    'Congo':'Republic of the Congo',
                    'Taiwan, Republic of China':'Taiwan',
                    'Turkiye':'Turkey',
                 }
ohnsmerger.replace_country_names(replacedict)
merge, unmatched = ohnsmerger.merge_data()
unmatched

NB Make sure zero values represent actual data, not missing data. They will be represented as zero on the map.


,Country Name,NE_Fuzzy_Match,NE_ISO


In [14]:
merge[merge['Oral, Head, & Neck Surgeons']== 0][['NAME_LONG','Oral, Head, & Neck Surgeons','Oral, Head & Neck Surgeons per Capita']]

,NAME_LONG,"Oral, Head, & Neck Surgeons","Oral, Head & Neck Surgeons per Capita"
156,The Gambia,0.0,0.0
178,Kiribati,0.0,0.0


In [15]:
merge[['NAME_LONG','geometry','Oral, Head, & Neck Surgeons','Oral, Head & Neck Surgeons per Capita']].pipe(save_geojson_versions,'ohns')

Uncompressed GeoJSON saved to static/geojson/ohns.geojson
Compressed GeoJSON saved to static/geojson/ohns.geojson.gz


# GAWS (class)

In [16]:
gawsmerger = DataMerger(data_110, data_50, 'data/dfg.csv',iso_col='code3', country_col='country')

## These locations are missing an ISO code in the GAWS data. Add them manually so they can be matched with Natural Earth.
filldict = {
    'Niue':'NIU',
    'Taiwan, Republic of China':'TWN',
    'Tokelau':'TKL',
    'Somaliland':'SOL',
}

for i in filldict.keys():
    gawsmerger.data.loc[gawsmerger.data['country']==i,'code3'] = filldict[i]

# These locations have a mismatched ISO code in the GAWS data. Replace them with the correct code.
replacedict = {
    'PSE':'PSX',
}

gawsmerger.replace_country_names(replacedict)

# We are using NAME_LONG to show the data in the map. Natural Earth shows these separately. 
# Extract Denmark and Greenland geometries
denmark_geom = gawsmerger.NE_110.loc[gawsmerger.NE_110['NAME_LONG'] == 'Denmark', 'geometry'].values[0]
greenland_geom = gawsmerger.NE_110.loc[gawsmerger.NE_110['NAME_LONG'] == 'Greenland', 'geometry'].values[0]

# Perform the union of Denmark and Greenland geometries
new_denmark_geom = denmark_geom.union(greenland_geom)

# Update Denmark geometry in the GeoDataFrame
gawsmerger.NE_110.loc[gawsmerger.NE_110['NAME_LONG'] == 'Denmark', 'geometry'] = new_denmark_geom

# Remove Greenland entry from the GeoDataFrame
gawsmerger.NE_110 = gawsmerger.NE_110[gawsmerger.NE_110['NAME_LONG'] != 'Greenland']

merge, unmatched = gawsmerger.merge_data()
unmatched

NB Make sure zero values represent actual data, not missing data. They will be represented as zero on the map.


,code3,country,NE_Fuzzy_Match,NE_ISO
140,TKL,Tokelau,None,NaN


In [17]:
show(merge[['NAME','NAME_LONG','SOVEREIGNT','country','code3','ADM0_A3']])

## Fill in missing populations

In [18]:
#set the merged index to iso code
merge=merge.set_index('ADM0_A3')
print('number of countries:', len(merge), '\nnumber missing population:',len(merge.loc[merge['population'].isna()]))

# get world bank population data
pop = pd.read_csv('data/wbpop.csv').set_index('Country Code')
pop = pop[pop.index.isin(merge.index)].rename(columns={'2021':'population'})[['population']] # keep only population data for countries in the merged data
pop['population'] = pop['population']/100000
merge=merge.combine_first(pop)
print(merge[merge['population'].isna()][['NAME_LONG','population']])
print('number of countries',len(merge),'\nnumber missing population: ', len(merge.loc[merge['population'].isna()]))
merge=merge.reset_index()

number of countries: 190 
number missing population: 35
                               NAME_LONG  population
ATA                           Antarctica         NaN
ATF  French Southern and Antarctic Lands         NaN
CYN                      Northern Cyprus         NaN
FLK          Falkland Islands / Malvinas         NaN
SAH                       Western Sahara         NaN
SDS                          South Sudan         NaN
number of countries 190 
number missing population:  6


Checking for null vs nan values

In [19]:
show(merge[merge['totalnpap']==0][['NAME_LONG','totalpap','totalnpap','physicians2015','totalpap_cap','physicians2015_cap','population','nurses2015','nurses2015_cap','totalnpap_cap']])

In [20]:
merge[['NAME_LONG','geometry','totalpap','totalnpap','physicians2015','totalpap_cap','physicians2015_cap','population','nurses2015','nurses2015_cap','totalnpap_cap']].pipe(save_geojson_versions, 'gaws_merged')

Uncompressed GeoJSON saved to static/geojson/gaws_merged.geojson
Compressed GeoJSON saved to static/geojson/gaws_merged.geojson.gz


## prepare for download

In [21]:
(merge[['index','NAME_LONG','population','totalpap','totalpap_cap','totalnpap','totalnpap_cap','physicians2015','physicians2015_cap','nurses2015','nurses2015_cap',]]
 .rename(columns={
     'index':'iso3',
     'poulation': 'population (2021)',
     'NAME_LONG':'country',
     'totalpap':'PAPs',
     'totalnpap':'NPAPs',
     'physicians2015':'PAPs (2016)',
     'totalpap_cap':'PAP density',
     'physicians2015_cap':'PAP density (2016)',
     'nurses2015':'Nurses (2016)',
     'nurses2015_cap':'Nurse density (2016)',
     'totalnpap_cap':'NPAP density'})).to_csv('static/raw/gaws.csv', index=False)